In [117]:
import pandas as pd


# In[83]:
df = pd.read_csv('Data_Train.csv')
df.head(10)
print(df.shape)
import spacy


(78458, 11)


In [118]:

#Replace K, M, B
def value_to_float(x):
    if type(x) == float or type(x) == int:
        return x
    if 'K' in x:
        if len(x) > 1:
            return float(x.replace('K', '')) * 1000
        return 1000.0
    if 'M' in x:
        if len(x) > 1:
            return float(x.replace('M', '')) * 1000000
        return 1000000.0
    if 'B' in x:
        return float(x.replace('B', '')) * 1000000000
    if ',' in x:
        return float(x.replace(',', '')) 
    return float(x)

df.Likes = df.Likes.apply(value_to_float)
df.Popularity = df.Popularity.apply(value_to_float)
df.to_csv('Data_Train_Transformed_1.csv')
df = df[df['Views'] > df["Likes"]]

In [119]:
from spacy.lang.en import English
nlp = English()
from spacy.lang.en.stop_words import STOP_WORDS
def remove_stop_words(song_name,name) :

    song_name = str(song_name)
    song_name = song_name.strip(str(name))
    doc = nlp(song_name)
    new_song_name = []
    for tokens in doc:
        if tokens.is_stop==False :
            new_song_name.append(tokens.text)
    return  (' '.join(new_song_name) )

df['new_song_name'] = df.apply(lambda x : remove_stop_words(x['Song_Name'],x['Name']),axis=1)

In [55]:
print(df.shape)

(77170, 12)


In [120]:
import sklearn as sklearn
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
# ngram level tf-idf 
tfidf_vect_ngram = TfidfVectorizer(analyzer='word',  ngram_range=(1,2),max_features=400)
fitted_vec = tfidf_vect_ngram.fit_transform(df['new_song_name'].values.astype('U'))

df_song_name = pd.DataFrame(fitted_vec.toarray(), columns=tfidf_vect_ngram.get_feature_names())


In [141]:
df.head()

,Unique_ID,Name,Genre,Country,Song_Name,Timestamp,Views,Comments,Likes,Popularity,Followers,new_song_name
0,413890,Hardstyle,danceedm,AU,N-Vitral presents BOMBSQUAD - Poison Spitter (...,2018-03-30 15:24:45.000000,14017,4,499.0,97.0,119563,N - Vitral presents BOMBSQUAD - Poison Spitter...
1,249453,Dj Aladdin,danceedm,AU,Dj Aladdin - Old School Hip Hop Quick Mix,2016-06-20 05:58:52.000000,1918,17,49.0,17.0,2141,- Old School Hip Hop Quick Mix
2,681116,Maxximize On Air,danceedm,AU,Maxximize On Air - Mixed by Blasterjaxx - Epis...,2015-05-08 17:45:59.000000,9668,11,312.0,91.0,22248,- Mixed Blasterjaxx - Episode # 048
3,387253,GR6 EXPLODE,rbsoul,AU,MC Yago - Tenho Compromisso (DJ R7),2017-06-08 23:50:03.000000,113036,2,2400.0,76.0,393655,MC Yago - Tenho Compromisso ( DJ R7 )
4,1428029,Tritonal,danceedm,AU,Escape (feat. Steph Jones),2016-09-17 20:50:19.000000,110024,81,3031.0,699.0,201030,Escape ( feat . Steph Jones )


In [148]:
df_name_avg_likes = df.groupby('Name')['Likes'].mean()
df['Avg_likes'] = df['Name'].map(df_name_avg_likes)

Name
$UICIDEBOY$          59362.255952
- S E C K O M -       2517.000000
12th Planet           6667.191489
16yrold             195764.000000
1788-L               10576.333333
                        ...      
ＳＨＥeＳＨ               19130.571429
🌸fatboibari🌸          3715.565217
🍺 Hiruko 🍺            1067.000000
👻 Hi I'm Ghost 👻      2561.523810
󠀠                     3094.426230
Name: Likes, Length: 1208, dtype: float64

In [150]:
df.head()

,Unique_ID,Name,Genre,Country,Song_Name,Timestamp,Views,Comments,Likes,Popularity,Followers,new_song_name,Avg_likes
0,413890,Hardstyle,danceedm,AU,N-Vitral presents BOMBSQUAD - Poison Spitter (...,2018-03-30 15:24:45.000000,14017,4,499.0,97.0,119563,N - Vitral presents BOMBSQUAD - Poison Spitter...,773.388790
1,249453,Dj Aladdin,danceedm,AU,Dj Aladdin - Old School Hip Hop Quick Mix,2016-06-20 05:58:52.000000,1918,17,49.0,17.0,2141,- Old School Hip Hop Quick Mix,285.541667
2,681116,Maxximize On Air,danceedm,AU,Maxximize On Air - Mixed by Blasterjaxx - Epis...,2015-05-08 17:45:59.000000,9668,11,312.0,91.0,22248,- Mixed Blasterjaxx - Episode # 048,446.406250
3,387253,GR6 EXPLODE,rbsoul,AU,MC Yago - Tenho Compromisso (DJ R7),2017-06-08 23:50:03.000000,113036,2,2400.0,76.0,393655,MC Yago - Tenho Compromisso ( DJ R7 ),4208.925714
4,1428029,Tritonal,danceedm,AU,Escape (feat. Steph Jones),2016-09-17 20:50:19.000000,110024,81,3031.0,699.0,201030,Escape ( feat . Steph Jones ),2257.355634


In [151]:
#Correlation matrix
df.drop(['Unique_ID','Genre','Country','Timestamp','Song_Name','Followers','Comments','Popularity','new_song_name'],inplace = True, axis = 1)


   Unique_ID              Name     Genre Country  \
0     413890         Hardstyle  danceedm      AU   
1     249453        Dj Aladdin  danceedm      AU   
2     681116  Maxximize On Air  danceedm      AU   
3     387253       GR6 EXPLODE    rbsoul      AU   
4    1428029          Tritonal  danceedm      AU   

                                           Song_Name  \
0  N-Vitral presents BOMBSQUAD - Poison Spitter (...   
1          Dj Aladdin - Old School Hip Hop Quick Mix   
2  Maxximize On Air - Mixed by Blasterjaxx - Epis...   
3                MC Yago - Tenho Compromisso (DJ R7)   
4                         Escape (feat. Steph Jones)   

                    Timestamp   Views  Comments   Likes  Popularity  \
0  2018-03-30 15:24:45.000000   14017         4   499.0        97.0   
1  2016-06-20 05:58:52.000000    1918        17    49.0        17.0   
2  2015-05-08 17:45:59.000000    9668        11   312.0        91.0   
3  2017-06-08 23:50:03.000000  113036         2  2400.0        76.

,Name,Views,Likes,Avg_likes
0,Hardstyle,14017,499.0,773.388790
1,Dj Aladdin,1918,49.0,285.541667
2,Maxximize On Air,9668,312.0,446.406250
3,GR6 EXPLODE,113036,2400.0,4208.925714
4,Tritonal,110024,3031.0,2257.355634


In [153]:
# from sklearn.preprocessing import LabelEncoder,OneHotEncoder
# # Converting Genre and Name to labelencoding
# labelencoder = LabelEncoder()
# df['Name'] = labelencoder.fit_transform(df['Name'])
# df.head()

import numpy as np
bins = [-1,0.1,4,10,20,30,50,75,100,125,150,175,200,230,270,300,330,370,400,450,500,600,650,700,750,800,850,950,1000,1200,1400,1600,
        1800,2000,2300,2700,3000,3500,4000,5000,6000,7000,8000,10000,15000,
        20000,40000,60000,100000,np.inf]
names = [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,
         38,39,40,41,42,43,44,45,46,47,48,49]
df['Name_Label'] = pd.cut(df['Avg_likes'], bins, labels=names)


In [156]:
df.drop(['Name','Avg_likes'],inplace = True, axis = 1)
df.head()

,Views,Likes,Name_Label
0,14017,499.0,25
1,1918,49.0,15
2,9668,312.0,19
3,113036,2400.0,39
4,110024,3031.0,34


In [158]:
df.drop(['Name','Avg_likes'],inplace = True, axis = 1)
df.head()
#Standard scaling
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
sc_x = StandardScaler()
ct = ColumnTransformer([
         ('somename', StandardScaler(), ['Likes'])
       ], remainder='passthrough')
df = ct.fit_transform(df)
df = pd.DataFrame(df,columns=['Likes','Views','Name_Label'])
print(df.head())

      Likes   Views Name_Label
0 -0.159204   14017         25
1 -0.167662    1918         15
2 -0.162719    9668         19
3 -0.123471  113036         39
4  -0.11161  110024         34


In [163]:
# df = df.join(name_hashed_features)
df = df.join(df_song_name)
print(df.shape)
df['Likes'] = df['Likes'].apply(pd.to_numeric, errors='coerce')
df['Name_Label'] = df['Name_Label'].apply(pd.to_numeric, errors='coerce')

In [164]:
y = df['Views']
df.drop(['Views'],inplace=True,axis=1)
y.plot.hist()

KeyError: 'Views'

In [165]:
from sklearn.model_selection import train_test_split
#print(df_scaled)
# split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(df, y, test_size=.2, random_state=205)

In [166]:
#XGB Boost regression
import xgboost as xgb
from sklearn import metrics
xlf = xgb.XGBRegressor()
xlf.fit(X_train, y_train, eval_metric='rmse', verbose = True, eval_set = [(X_test, y_test)])
# calculate the auc score
y_pred = xlf.predict(X_test)
#RMSE - Test
print("RMSE - XGBoost regression: ", np.sqrt(metrics.mean_squared_error(y_test,y_pred)))


d:\applnsanaconda3\envs\hackathon\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


[12:09:16] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[0]	validation_0-rmse:3.35707e+06
[1]	validation_0-rmse:3.03555e+06
[2]	validation_0-rmse:2.75289e+06
[3]	validation_0-rmse:2.50845e+06
[4]	validation_0-rmse:2.27444e+06
[5]	validation_0-rmse:2.0732e+06
[6]	validation_0-rmse:1.89272e+06
[7]	validation_0-rmse:1.72244e+06
[8]	validation_0-rmse:1.58427e+06
[9]	validation_0-rmse:1.45174e+06
[10]	validation_0-rmse:1.3412e+06
[11]	validation_0-rmse:1.23859e+06
[12]	validation_0-rmse:1.14503e+06
[13]	validation_0-rmse:1.06323e+06
[14]	validation_0-rmse:990654
[15]	validation_0-rmse:929112
[16]	validation_0-rmse:873369
[17]	validation_0-rmse:828192
[18]	validation_0-rmse:788828
[19]	validation_0-rmse:753010
[20]	validation_0-rmse:725610
[21]	validation_0-rmse:699107
[22]	validation_0-rmse:678601
[23]	validation_0-rmse:656746
[24]	validation_0-rmse:641041
[25]	validation_0-rmse:62701

In [192]:
import pandas as pd
# In[83]:
test_df = pd.read_csv('Data_Test.csv')
test_df.head(10)
print(test_df.shape)

(19615, 10)


In [193]:
#Replace K, M, B
def value_to_float(x):
    if type(x) == float or type(x) == int:
        return x
    if 'K' in x:
        if len(x) > 1:
            return float(x.replace('K', '')) * 1000
        return 1000.0
    if 'M' in x:
        if len(x) > 1:
            return float(x.replace('M', '')) * 1000000
        return 1000000.0
    if 'B' in x:
        return float(x.replace('B', '')) * 1000000000
    if ',' in x:
        return float(x.replace(',', '')) 
    return float(x)

test_df.Likes = test_df.Likes.apply(value_to_float)
test_df.Popularity = test_df.Popularity.apply(value_to_float)

In [194]:
test_df['new_song_name'] = test_df.apply(lambda x : remove_stop_words(x['Song_Name'],x['Name']),axis=1)

In [195]:
test_df.head()

,Unique_ID,Name,Genre,Country,Song_Name,Timestamp,Comments,Likes,Popularity,Followers,new_song_name
0,562546,L2Share♫79,all-music,AU,엔플라잉 N.Flying - Rooftop (옥탑방),2019-01-02 09:49:19.000000,26,5428.0,118.0,2568,엔플라잉 N.Flying - Rooftop ( 옥탑방 )
1,907584,Morgan Page,danceedm,AU,Morgan Page - In The Air - Episode 246,2015-03-09 17:30:10.000000,18,738.0,195.0,155645,- Air - Episode 246
2,213013,Dirty Workz,danceedm,AU,Outlander - Devotion (Official HQ Preview),2013-09-18 16:09:03.000000,18,386.0,96.0,29446,Outlander - Devotion ( Official HQ Preview )
3,340312,Gucci Mane,all-music,AU,Freaky Gurl,2007-08-14 00:00:00.000000,1,388.0,19.0,799410,Freaky Gurl
4,41854,Angga Kecap,rock,AU,Radical Nusantara - Teman,2018-08-10 16:28:54.000000,0,79.0,1.0,2,Radical Nusantara - Tem


In [196]:
#Correlation matrix
test_fitted_vec = tfidf_vect_ngram.transform(test_df['new_song_name'].values.astype('U'))
test_df_song_name = pd.DataFrame(test_fitted_vec.toarray(), columns=tfidf_vect_ngram.get_feature_names())

In [197]:
#Correlation matrix
test_df.drop(['Unique_ID','Genre','Country','Timestamp','Popularity','Followers','Comments','Song_Name','new_song_name'],inplace = True, axis = 1)

In [198]:
test_df.shape

(19615, 2)

In [199]:
# from sklearn.preprocessing import LabelEncoder,OneHotEncoder
# # Converting Genre and Name to labelencoding
# labelencoder = LabelEncoder()
# test_df['Name'] = labelencoder.fit_transform(test_df['Name'])
test_df['Avg_likes'] = test_df['Name'].map(df_name_avg_likes)
test_df.head()

,Name,Likes,Avg_likes
0,L2Share♫79,5428.0,4009.125000
1,Morgan Page,738.0,808.487342
2,Dirty Workz,386.0,444.341040
3,Gucci Mane,388.0,14868.724551
4,Angga Kecap,79.0,42.000000


In [200]:
import numpy as np
bins = [-1,0.1,4,10,20,30,50,75,100,125,150,175,200,230,270,300,330,370,400,450,500,600,650,700,750,800,850,950,1000,1200,1400,1600,
        1800,2000,2300,2700,3000,3500,4000,5000,6000,7000,8000,10000,15000,
        20000,40000,60000,100000,np.inf]
names = [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,
         38,39,40,41,42,43,44,45,46,47,48,49]
test_df['Name_Label'] = pd.cut(test_df['Avg_likes'], bins, labels=names)

In [202]:
test_df.drop(['Name','Avg_likes'],inplace = True, axis = 1)
#Standard scaling
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
sc_x = StandardScaler()
ct = ColumnTransformer([
         ('somename', StandardScaler(), ['Likes'])
       ], remainder='passthrough')
test_df = ct.fit_transform(test_df)
test_df = pd.DataFrame(test_df,columns=['Likes','Name_Label'])
print(test_df.head())

KeyError: "['Name' 'Avg_likes'] not found in axis"

In [203]:
test_df = test_df.join(test_df_song_name)
print(test_df.shape)
test_df['Likes'] = test_df['Likes'].apply(pd.to_numeric, errors='coerce')
# df['Popularity'] = df['Popularity'].apply(pd.to_numeric, errors='coerce')

(19615, 402)


In [205]:
# calculate the auc score
y_pred_1 = xlf.predict(test_df)
y_pred_1 = pd.DataFrame(y_pred_1)
y_pred_1.to_csv('Data_Test_PredictedSong_Name_Label_Encode_Artist_Name_Likes.csv',sep='#')